In [1]:
# !pip install transformers
# !pip install bertviz
# !pip install pytorch

In [2]:
# Text example (For use throughout notebook)

text = "I am a big fan of pugs"

In [3]:
from transformers import AutoTokenizer
from bertviz.transformers_neuron_view import BertModel
from bertviz.neuron_view import show

In [5]:
# Visualizing the concept of self-attention and how it works

model_ckpt = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = BertModel.from_pretrained(model_ckpt)
show(model, "bert", tokenizer, text, display_mode="light", layer=0, head=0)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
# Tokenize our input text
inputs = tokenizer(text, return_tensors="pt", add_special_tokens=False)
inputs.input_ids

tensor([[ 1045,  2572,  1037,  2502,  5470,  1997, 16405,  5620]])

In [12]:
# Get all the token embeddings available in the Bert Model vocabulary

from torch import nn
from transformers import AutoConfig

config = AutoConfig.from_pretrained(model_ckpt)
token_emb = nn.Embedding(config.vocab_size, config.hidden_size)
token_emb

Embedding(30522, 768)

In [14]:
# Creating token embeddings for our text

input_embeds = token_emb(inputs.input_ids)
input_embeds.size()

torch.Size([1, 8, 768])

In [22]:
# Simplified implementation of the self-attention
# For conceptualization only not for practice

import torch
from math import sqrt

# Multiple our query and keys matrices, and scale them
query = key = value = input_embeds
dim_k = key.size(-1)
scores = torch.bmm(query, key.transpose(1,2)) / sqrt(dim_k)
print(scores.size())

# Apply softmax to output
import torch.nn.functional as F

weights = F.softmax(scores, dim=1)
print(weights.sum(dim=-1))

# Multiply the outputs by our value matrix
attn_outputs = torch.bmm(weights, value)
print(attn_outputs.shape)

torch.Size([1, 8, 8])
tensor([[1., 1., 1., 1., 1., 1., 1., 1.]], grad_fn=<SumBackward1>)
torch.Size([1, 8, 768])


In [23]:
# Now let's wrap that logic into a function that we can use for self-attention later

def sclaed_dot_product_attention(query, key, value):
    dim_k = query.size(-1)
    scores = torch.bmm(query, key.transpose(1,2)) / sqrt(dim_k)
    weights = F.softmax(scores, dim=1)
    attn_outputs = torch.bmm(weights, value)
    return attn_outputs

In [ ]:
# Implementation of a single Attention Head

class AttentionHead(nn.Module):
    def __init__(self, embed_dim, head_dim):
        super.__init__()
        self.q = nn.Linear(embed_dim, head_dim)
        self.k = nn.Linear(embed_dim, head_dim)
        self.v = nn.Linear(embed_dim, head_dim)
        
    def forward(self, hidden_state):
        attn_outputs = scaled_dot_product_attention(
        self.q(hidden_state), self.k(hidden_state), self.v(hidden_state)
        )
        return attn_outputs